# UCB CE170A Assignment 1 : Structural Analysis of a Model Bridge using a Python code
v2 - Fall 2020: Prof.Kenichi Soga, Yaobin Yang, Renjie Wu  

# <font color=”#8B0000”> Save the file to your google drive first! (Click Copy to Drive) </font> 

# Install required packages

In [ ]:
!pip install trusspy

# Background

<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw1/images/figure0.png" alt="A bridge." width="400" />
    
    An example bridge
</center>


In the sensor module scheduled later on in the semester, we will be instrumenting a model bridge and testing it to failure. A photo and schematic diagram of the model bridge is shown in Fig. 1. The model consists of two identical 2D frames placed in parallel and connected by diagonal and horizonal braces (Pink lines). A vertical load will be applied at the top two nodes using a load spreader frame (light Blue). In the experiment, the model bridge will be placed between two rigid supports as shown in Fig. 2. 




<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw1/images/figure1.png" alt="schematic diagram." width="600" />
    
    Figure 1. A photo and schematic diagram of the model bridge
</center>

<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw1/images/figure2.png" alt="A bridge." width="500" />
    
    Figure 2. Loading on the model bridge
</center>

Run cell below to see the video:

In [2]:
from IPython.display import HTML
# Youtube
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/rdycElfVe4Y" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

The bridge is made of stainless steel angle bars and sheets as shown in Fig. 3. The angle bar has a cross-sectional dimension of ½ inch x ½ inch x 3/80 inch (thickness)(cross-sectional area = 2.33 x 10$^{-5}$ m$^{2}$). Two angle bars are used for the top four frame members (Yellow lines in Fig. 1 or L2 & L3 in Fig. 2), whereas one angle bar is used for the other members (Red, Blue and Pink lines in Fig. 1 or L4 & L5 in Fig. 2). The Young’s modulus and yield strength of the angle bar are 190 GPa and 350 MPa, respectively. The members are connected by rivets. 

The design details of the model bridge are given in Fig. 4.


<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw1/images/figure3.png" alt="A bridge." width="400" />
    
    Figure 3.  Steel angle bars and sheets used by the bridge
</center>


<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw1/images/figure4.png" alt="A bridge." width="600" />
    
    Figure 4. Design details of the model
</center>





# Quiz 1:
A vertical working load of W=3500 N is applied at the top section of the model structure. Considering the symmetry and using a simplified 2D model shown in Fig. 5, perform a hand calculation of the bar forces and strains under the given load for load carrying sections, L1, L2 and L3. What is the role of bracing sections L4 and L5? Estimate the displacement at the top node. (use dimensions in Figure 5 for calculations)

<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw1/images/figure5.png" alt="A bridge." width="600" />
    
    Figure 5. A simplified 2D model of the model bridge (dimension in meters)
</center>

# Question 1:
Following code blocks below to use [Trusspy](https://adtzlr.github.io/trusspy/) to model the 3D model bridge. Run all the cells and **fill the TODO** parts if necessary. 


In [ ]:
"""
Step 0: Import libraries  
"""
import trusspy as tp
import copy
import matplotlib.pyplot as plt
import numpy as np

First, lets declare constant variables and initialize the Trusspy model. Fill the empty constants using the information provided by the question description. 

In [ ]:
"""
Step 1: Assign Constant and initialze the model 
"""

# LOAD = 3.5e3   # Total Load [N] 
ELEMENT_TYPE   = 1    # truss
MATERIAL_TYPE  = 2    # elasto-plastic
HARDENING_MODULUS = 0.01 # hardening_modulus[Pa] a very small number for elastic-perfectly plastic
A = 2.33e-5 # the area of the angle section [m^2]
INCREMENT = 30

#TODO: Assign proper value for the load below
YONGS_MODULUS =  # Young's modulus [Pa]
YIELD_STRENGH =  # yield strength[Pa]

# model settings
M_perfect = tp.Model(log = 0)# Trusspy model initialization 
M_perfect.Settings.dlpf = 0.1
M_perfect.Settings.du = 0.002
M_perfect.Settings.incs = INCREMENT
M_perfect.Settings.stepcontrol = True


Second, lets add nodes into the initialized null model. Model configuration is shown be the figure below: 

<center>
<img src="https://github.com/UCB-CE170a/Fall2020/raw/master/homeworks/hw1/images/model_configuration.png" alt="A bridge." width="600" />
    
    Figure 6. Trusspy model configuration 
</center>


In [ ]:
"""
Step 2: Add truss elements to the model 
"""
with M_perfect.Nodes as MN:# nodes defination unit [m]
    MN.add_node( 1, coord=(0,0,0))
    MN.add_node( 2, coord=(0.5,0,0))
    MN.add_node( 3, coord=(1,0,0))
    MN.add_node( 4, coord=(0.25,0,0.25))
    MN.add_node( 5, coord=(0.75,0,0.25))
    MN.add_node( 6, coord=(0.5,0,0.5))
    MN.add_node( 7, coord=(0,0.1438,0))
    MN.add_node( 8, coord=(0.5,0.1438,0))
    MN.add_node( 9, coord=(1,0.1438,0))
    MN.add_node( 10, coord=(0.25,0.1438,0.25))
    MN.add_node( 11, coord=(0.75,0.1438,0.25))
    MN.add_node( 12, coord=(0.5,0.1438,0.5))

Then we link all these nodes together to construct the configuration shown by figure 6. 

In [ ]:
"""
Step 3: Add truss elements to the model 
"""
with M_perfect.Elements as ME: 
    ME.add_element( 1, conn=(1,2), gprop=[A] )
    ME.add_element( 2 ,conn=(2,3), gprop=[A] )
    ME.add_element( 3, conn=(1,4), gprop=[A*2] )
    ME.add_element( 4, conn=(2,4), gprop=[A] )
    ME.add_element( 5, conn=(2,5), gprop=[A] )
    ME.add_element( 6, conn=(3,5), gprop=[A*2] )
    ME.add_element( 7, conn=(4,6), gprop=[A*2] )
    ME.add_element( 8, conn=(5,6), gprop=[A*2] )
    ME.add_element( 9, conn=(7,8), gprop=[A] )
    ME.add_element( 10 ,conn=(8,9), gprop=[A] )
    ME.add_element( 11, conn=(7,10), gprop=[A*2] )
    ME.add_element( 12, conn=(8,10), gprop=[A] )
    ME.add_element( 13, conn=(8,11), gprop=[A] )
    ME.add_element( 14, conn=(9,11), gprop=[A*2] )
    ME.add_element( 15, conn=(10,12), gprop=[A*2] )
    ME.add_element( 16, conn=(11,12), gprop=[A*2] )
    ME.add_element( 17, conn=(1,10), gprop=[A] )
    ME.add_element( 18, conn=(4,12), gprop=[A] )
    ME.add_element( 19, conn=(6,11), gprop=[A] )
    ME.add_element( 20, conn=(5,9), gprop=[A] )
    ME.add_element( 21, conn=(6,12), gprop=[A] )
    ME.add_element( 22, conn=(6,12), gprop=[A] )
    
    #TODO: Connect four more node pairs to complete the model 
    ME.add_element()
    ME.add_element()
    ME.add_element()
    ME.add_element()
    
    # assign material types to the model
    ME.assign_etype(    'all',   ELEMENT_TYPE   )
    ME.assign_mtype(    'all',  MATERIAL_TYPE   )
    ME.assign_material( 'all', [YONGS_MODULUS,HARDENING_MODULUS,YIELD_STRENGH] )

Lets assign external forces and boundary conditions to the model:

In [ ]:
"""
Step 4: Set boundary conditions and external forces
"""
#TODO: Fill the load 
load = 
load1=-0.5*load # front force
load2=-0.5*load # back force

with M_perfect.Boundaries as MB: # boundary-displacement
    MB.add_bound_U( 1, (0,0,0) )
    MB.add_bound_U( 3, (1,1,0) )
    MB.add_bound_U( 7, (0,1,0) )
    MB.add_bound_U( 9, (1,1,0) )
    MB.add_bound_U( 2, (1,0,1) ) # this is a difference between our model and realistic word, since there is no joint in the real world for node 2 and 8
    MB.add_bound_U( 8, (1,0,1) ) # since there is no joint in the real world for node 2 and 8
    
with M_perfect.ExtForces as MF: # boundary-force
    MF.add_force( 6, ( 0, 0, load1) )
    MF.add_force( 12, ( 0, 0, load2) )


Finally, lets build and run the model:

In [ ]:
"""
Step 5: Build and run the model
"""
M_perfect.build()
M_perfect.run()
print ("Simulation finished")


If your model is built successfully, you will see "Simulation finished" on the bottom of the above cell. Run cell below to plot the model in 3D and compare to Figure 6. Make sure they are identical before proceeding.


In [ ]:
# View the initial model in 3D
fig, ax = M_perfect.plot_model(config=['undeformed'],
                      view='3d', #'xy', 'yz', 'xz'
                      contour='force',
                      lim_scale=(-0.1,1.1,0,0.25,-0.1,0.5), #3d
                      #lim_scale=1.4, #plane-view
                      force_scale=0.00005, #2
                      inc=0)

Trusspy allows us to view the deformation in 3D: 

In [ ]:
  fig, ax = M_perfect.plot_model(config=['deformed'],
                        view='3d', #'xy', 'yz', 'xz'
                        contour='force',
                        lim_scale=(-0.1,1.1,0,0.25,-0.1,0.5), #3d
                        #lim_scale=1.4, #plane-view
                        force_scale=0.00005, #2
                        inc=-1)

And in 2d views:

In [ ]:
# "Deformed model in xz plane view"
fig, ax = M_perfect.plot_model(config=['deformed'],
                        view='xz',
                        contour='force',
                        lim_scale=1.3,
                        force_scale=0.00005,
                        inc=-1)

# Deformed model in xy plane'
fig, ax = M_perfect.plot_model(config=['deformed'],
                        view='xy',
                        contour='force',
                        lim_scale=1.3,
                        force_scale=0.00005,
                        inc=-1)

## Plot elements responses during loading. 


We are going to create some help functions to assess elements responses of the model during the loading process. Again, run all the cells and **fill the TODO** parts if necessary. 

First, lets create a function to get the internal forces of a truss element with respect to the load proportionality factor (LPFs).  

LPF = applied load/design load (which is 3500N in this case) 

In [ ]:
def get_internal_forces(M,elem): 
  """Get internal force of a element
    Args:
      M: trusspy model 
      elem: the element index of interest

    Returns:
      lpfs is a list of loading proportionality factor for this elem
      forces is a list of internal force for this elem regarding to lpfs
  """
  Res=M.Results.R
  lpfs,forces = [],[]
  for r in Res:
      force =r.element_force[elem-1]# element names is 1 based whereas list index is 0 based
      lpfs.append(r.lpf)
      forces.append(force[0])
  return lpfs,forces


Using the extracted force to create a function that computes stresses of a element regarding to LPFs: 

In [ ]:
def get_stresses(M,elem):
  """Get stress of a element
    Args:
      M: trusspy model 
      elem: the element index of interest

    Returns:
      lpfs is a list of loading proportionality factor for this elem
      stresses is a list of stress for this elem regarding to lpfs
  """
  area = M.Elements.geometric_properties[elem-1][0]# element names is 1 based whereas list index is 0 based
  lpfs,forces = get_internal_forces(M,elem)

  # TODO: Compute stresses from the internal forces
  stresses = 
  return lpfs,stresses

Now, lets compute strains. 

In [ ]:
def get_strains(M,elem): 
  """Get strain of a element
    Args:
      M: trusspy model 
      elem: the element index of interest

    Returns:
      lpfs is a list of loading proportionality factor for this elem
      strains is a list of strain for this elem regarding to lpfs
  """
  E=M.Elements.material_properties[elem-1][0]
  lpfs,stresses = get_stresses(M,elem)

  # TODO: Compute strains from the internal forces
  strains=
  return lpfs,strains

And rotation degrees of a element. 

In [ ]:
def get_rotation_degrees(M,elem):
  """Get rotation degrees of a element
    Args:
      M: trusspy model 
      elem: the element index of interest

    Returns:
      lpfs is a list of loading proportionality factor for this elem
      degrees is a list of rotation degree for this elem regarding to lpfs
  """
  con=M.Elements.conns[elem-1] # element names is 1 based whereas list index is 0 based
  node1,node2 =con[0], con[1]
  coord1,coord2=M.Nodes.coords[node1-1], M.Nodes.coords[node2-1]# node names is 1 based whereas list index is 0 based
  r_before = coord2-coord1 #initial vector representation of the truss element

  Res=M.Results.R
  lpfs,degrees = [],[]
  for Re in Res:
      n1_displacement = Re.U[node1-1]
      n2_displacement = Re.U[node2-1]
        
      # TODO: Get the coordinates of the two nodes after stressing. 
      coord1_after = 
      coord2_after = 
      # TODO: Use all the information to get the rotation degree:theta 
      # Hint: How do you get the angle of two vector? np.dot might be helpful
      
      theta = 
      lpfs.append(Re.lpf)
      degrees.append(theta)
  return lpfs,degrees



Using the created plot the following with increasing load (Load Proportionality Ratios).

### Q1 a): Strains and rotations in element L2







What is the internal force and rotation of L2 when LPF is around 1? (use the closest LPF and the corresponding values). Compare values from simulation to your results in Quiz1. 

In [ ]:
#TODO code to get the value 

<font color=”blue”> Your Answer </font> : 

In [ ]:
#TODO strain plot
lpf,epis= 
plt.plot(lpf,epis,label='L2 strain')
plt.xlabel('LPF')
plt.ylabel('Strain')
plt.legend()


In [ ]:
#TODO rotation plot
lpf,thetas= 
plt.plot(lpf,thetas,label='L2 Rotation')
plt.xlabel('LPF')
plt.ylabel('Rotation/rad')
plt.legend()

### Q1 b): Expected expansion of element L1



Fill the function below to use strains to find expansions.

In [ ]:
def calculate_distance(coord1,coord2):
  """Calculate the distance of two points
    Args:
      coord1: coordinate of point (node) 1
      coord2: coordinate of point (node) 2

    Returns:
      L: distance between two points
    """
  #TODO: Calculate the distance of two points using numpy library
  L=
  return L


def get_expansion(M,elem):
  """Get expansion of a element
    Args:
      M: trusspy model 
      elem: the element index of interest

    Returns:
      lpfs is a list of loading proportionality factor for this elem
      expansions is a list of expansions for this elem regarding to lpfs
  """
  con=M.Elements.conns[elem-1] # element names is 1 based whereas list index is 0 based
  con1=con[0]
  con2=con[1]
  nc1=M.Nodes.coords[con1-1]
  nc2=M.Nodes.coords[con2-1]
    
  # TODO: Compute the expansion
  # HINT: Use the strain function    
  L=calculate_distance(nc1,nc2) # fill the function above to do the distance calculation

  expansions = 
  return lpfs,expansions


What is the expected expansion of L1 when LPF is around 1? (use the closest LPF and the corresponding values). Compare values from simulation to your results in Quiz1. 

In [ ]:
#TODO code to get the value 

<font color=”blue”> Your Answer </font> : 

In [ ]:
#TODO expansion plot
lpf,exp= 
plt.figure()
plt.plot(lpf,exp,label='L1 expansion')
plt.xlabel('LPF')
plt.ylabel('Expansion (m)')
plt.legend()

### Q1 c): Expected loads in braces: L4, L5 and Ts.

What is the expected loads of these braces when LPF is around 1? (use the closest LPF and the corresponding values). Compare values from simulation to your results in Quiz1. 

In [ ]:
#TODO codes to get the answer 


<font color=”blue”> Your Answer </font> : 

In [ ]:
#TODO L4 plots
lpf,force= 
plt.plot(lpf,force,label='L4 internal force')
plt.xlabel('LPF')
plt.ylabel('internal force/N')
plt.legend()

#TODO L5 plots
lpf,force= 
plt.plot(lpf,force,label='L5 internal force')
plt.xlabel('LPF')
plt.ylabel('internal force/N')
plt.legend()



In [ ]:
#TODO Ts plots
lpf,force= 
plt.plot(lpf,force,label='T1 internal force')
lpf,force= 
plt.plot(lpf,force,label='T2 internal force')
lpf,force= 
plt.plot(lpf,force,label='T3 internal force')
lpf,force= 
plt.plot(lpf,force,label='T4 internal force')
lpf,force= 
plt.plot(lpf,force,label='T5 internal force')
plt.xlabel('LPF')
plt.ylabel('internal force/N')
plt.legend()

## Question 2

This model bridge is designed to carry loads without collapse at this working load and the factor of safety to collapse is FS>2.0 (or when the load proportionality factor is greater than 2). Discuss possible failure mechanisms of the model bridge.

<font color=”blue”> Your Answer </font> : 